In [6]:
import mysql.connector
import pandas as pd
import getpass

In [101]:
#103.135.214.213
#analis_sirs
#P@ssw0rd2021!
mydb = mysql.connector.connect(
  host=input('host:'),
  user=input('user:'),
  password=getpass.getpass('password:'),
  database=getpass.getpass('db:') 
)

mycursor = mydb.cursor(dictionary=True)

host:103.135.214.213
user:analis_sirs
password:········
db:········


In [97]:
def exec(q):
  mycursor.execute(q)
  return mycursor.fetchall()

def exc_q_transfer(Y,dp='183',T='M'):
    dpT = 'NULL'
    dpK = dp
    if(T == 'M'):
        dpT = dpK
        dpK = 'NULL'
    return exec('SELECT \
        inventori_transfer_logistik_detail.no_transfer,inventori_transfer_logistik.tgl_acc_permintaan AS tgl_transfer ,\
        masterf_katalog.nama_barang,qty_diberi as qty_diberi,hna_ppn_transfer hna_ppn,masterf_katalog.hna,\
        masterf_katalog.harga_terakhir,\
        SUM(hna_ppn_transfer * qty_diberi) AS total, md1.namaDepo AS penerima, md2.namaDepo AS pengirim,\
            jadbf2143.username,md1.id,md2.id, nama_kode_satuan_kecil,uraian, kode_obat, inventori_transfer_logistik.keterangan\
        FROM inventori_transfer_logistik\
        LEFT JOIN inventori_transfer_logistik_detail \
        ON inventori_transfer_logistik.no_transfer = inventori_transfer_logistik_detail.no_transfer\
        LEFT JOIN `masterf_depo` as md1 ON `md1`.`id` = `inventori_transfer_logistik`.`gudang_tujuan`\
        LEFT JOIN masterf_katalog on inventori_transfer_logistik_detail.kode_obat=masterf_katalog.kode\
        LEFT JOIN `masterf_depo` as md2 ON `md2`.`id` = `inventori_transfer_logistik`.`gudang_asal`\
        LEFT JOIN `user` as jadbf2143 ON `jadbf2143`.`id` = `inventori_transfer_logistik`.`acc_oleh`\
        INNER JOIN master_satuan_kecil e ON id_kode_satuan_kecil=kodeSatuanKecil\
        INNER JOIN master_sumber_referensi on (inventori_transfer_logistik.status=kode_ref and tipe_ref = "IST")\
        WHERE  1 = 1  AND inventori_transfer_logistik.status = 2  AND \
        DATE_FORMAT(inventori_transfer_logistik.tgl_acc_permintaan,"%Y-%m-%d %H:%i") \
        BETWEEN "'+Y+'-01-01 00:00" AND "'+Y+'-12-31 23:59" \
        AND kode_obat =IFNULL(NULL,kode_obat)\
        AND `inventori_transfer_logistik`.`gudang_tujuan` = IFNULL('+dpT+',`inventori_transfer_logistik`.`gudang_tujuan`)\
        AND `inventori_transfer_logistik`.`gudang_asal` = IFNULL('+dpK+',`inventori_transfer_logistik`.`gudang_asal`)  \
        GROUP BY inventori_transfer_logistik_detail.id')

def do_in_out(Y,dp):
    R = []
    R.append(exc_q_transfer(Y,dp,'M'))
    R.append(exc_q_transfer(Y,dp,'K'))
    return R

def komparasiDenganDesc(D1,D2,r=0):
    return pd.merge(D1,D2,on=r,how='outer',indicator=True)

#bangun query
def bangun_q(DF,po,selisihM):
    trn = ''
    ret = ''
    for i,r in DF[['kode_obat','qty_diberi','no_transfer','keterangan']]\
[(DF['keterangan'].str.contains(pat=po,regex=True,case=False))].iterrows():
        ret += "update inventori_penerimaan_logistik_farmasi_detail set qty = qty - "+str(r['qty_diberi'])+" \
    where no_po = '"+po+"' and kode_obat = '"+r['kode_obat']+"';\n \
update log_stock_obatdepo set \
    jumlah_mutasi = jumlah_mutasi - "+str(r['qty_diberi'])+" where id_referensi = '"+po+"' \
    and kode_obat = '"+r['kode_obat']+"';\n"
        if(trn != r['no_transfer']):
            trn = r['no_transfer']
            ret += "UPDATE inventori_transfer_logistik\
 SET tgl_buat_permintaan = DATE_ADD(tgl_buat_permintaan, INTERVAL "+selisihM+" MONTH),\
tgl_acc_permintaan = DATE_ADD(tgl_acc_permintaan, INTERVAL "+selisihM+" MONTH) \
WHERE no_transfer = '"+trn+"';\n \
update log_stock_obatdepo set last_update = DATE_ADD(last_update, INTERVAL "+selisihM+" MONTH) \
where id_referensi = '"+trn+"';\n"
    return ret

In [144]:
R = do_in_out('2022','197')

OperationalError: 2055: Lost connection to MySQL server at '103.135.214.213:3306', system error: 10060 A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond

In [103]:
DTM = pd.DataFrame.from_dict(R[0])
DTK = pd.DataFrame.from_dict(R[1])
DTMG = DTM[['kode_obat','qty_diberi']].groupby(by='kode_obat').sum()
DTKG = DTK[['kode_obat','qty_diberi']].groupby(by='kode_obat').sum()
DC = komparasiDenganDesc(DTMG,DTKG,'kode_obat')
DC.loc[:,'agg'] = DC['qty_diberi_x'].fillna(0) - DC['qty_diberi_x'].fillna(0)
DC[(DC['agg'] > 0)]

,qty_diberi_x,qty_diberi_y,_merge,agg
kode_obat,,,,


In [143]:
print(bangun_q(DTK,'00115/PO/PPO/04/2022','-1'))

update inventori_penerimaan_logistik_farmasi_detail set qty = qty - 1.00     where no_po = '00115/PO/PPO/04/2022' and kode_obat = '020700163';
 update log_stock_obatdepo set     jumlah_mutasi = jumlah_mutasi - 1.00 where id_referensi = '00115/PO/PPO/04/2022'     and kode_obat = '020700163';
UPDATE inventori_transfer_logistik SET tgl_buat_permintaan = DATE_ADD(tgl_buat_permintaan, INTERVAL -1 MONTH),tgl_acc_permintaan = DATE_ADD(tgl_acc_permintaan, INTERVAL -1 MONTH) WHERE no_transfer = 'TRN/220510/208/197/00067';
 update log_stock_obatdepo set last_update = DATE_ADD(last_update, INTERVAL -1 MONTH) where id_referensi = 'TRN/220510/208/197/00067';
update inventori_penerimaan_logistik_farmasi_detail set qty = qty - 1.00     where no_po = '00115/PO/PPO/04/2022' and kode_obat = '020700221';
 update log_stock_obatdepo set     jumlah_mutasi = jumlah_mutasi - 1.00 where id_referensi = '00115/PO/PPO/04/2022'     and kode_obat = '020700221';



In [112]:
l = [2,3]
m = map(lambda x: 2**x,l)
print(list(m))

[4, 8]


In [113]:
a = [2,5J,6]
print(a.sort())

TypeError: '<' not supported between instances of 'complex' and 'int'

In [115]:
print([i.lower() for i in "TURING"])

['t', 'u', 'r', 'i', 'n', 'g']


In [118]:
a = [1,2,3]
b = [sum(a[0:x+1]) for x in range(0,len(a))]
print(b)

[1, 3, 6]


In [132]:
z=set('abc')
z.add('san')
z.update(set(['p','q']))
print(z)

{'p', 'c', 'q', 'b', 'a', 'san'}


In [137]:
'sff asas DDD'.capitalize()

'Sff asas ddd'

In [138]:
print([1,2] == [3,4])

False


In [142]:
'The {0} side {1}'.format('dd','dd1')

'The dd side dd1'

In [140]:
print([2,6])

[2, 6]
